# Data acquisition and basic manipulation

We're going to start with data from [covidtracking.com](https://covidtracking.com/).  This is an excellent resource that provides an API for curated, historical, state-by-state data.

In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.enable('json')

df=pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")

We're first going to reformat the dates to add hyphens between the year, month, and day, so `20200228` becomes `2020-02-28`.

In [2]:
def datemunge(di):
    d = str(di)
    return "%s-%s-%s" % (d[0:4], d[4:6], d[6:8])

cleaned = df.copy()

cleaned["date"] = cleaned["date"].apply(datemunge)

We'll then `melt` the data frame so that each observation is in its own row, so that (for example) `state, date, positive, negative, hospitalized, icu` becomes `state, date, observation_type, observation_value`, where `observation_type` is one of `positive`, `negative`, `hospitalized`, or `icu`.

In [3]:
cleaned = pd.melt(cleaned, 
                  id_vars=['date', 'state', 'fips'], 
                  value_vars=list(set(df.columns) - set(['date', 'state', 'fips', 'hash', 'dateChecked'])), 
                  value_name="cases",
                  var_name="case type")


We can see the difference between these representations by looking at the source data (`df`) for Wisconsin on April 9th and the melted data (`cleaned`) for Wisconsin on April 9th.

In [4]:
df[(df["state"] == "WI") & (df["date"] == 20200409)]

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
109,20200409,WI,2885.0,31424.0,NaN,NaN,843.0,NaN,230.0,NaN,...,843.0,34309,34309,34309,55,12.0,53.0,1309.0,129.0,1438.0


In [5]:
cleaned[(cleaned["state"] == "WI") & (cleaned["date"] == "2020-04-09")].dropna()

,date,state,fips,case type,cases
109,2020-04-09,WI,55,posNeg,34309.0
2109,2020-04-09,WI,55,death,111.0
4109,2020-04-09,WI,55,deathIncrease,12.0
6109,2020-04-09,WI,55,negative,31424.0
10109,2020-04-09,WI,55,totalTestResultsIncrease,1438.0
14109,2020-04-09,WI,55,hospitalizedCumulative,843.0
16109,2020-04-09,WI,55,hospitalized,843.0
18109,2020-04-09,WI,55,positive,2885.0
20109,2020-04-09,WI,55,positiveIncrease,129.0
24109,2020-04-09,WI,55,inIcuCumulative,230.0


# Per-state results

In [6]:
def cases_for_state(state, show_points=False):
    case_types = ['death', 'positive', 'hospitalizedCumulative', 'inIcuCumulative']
    chart = alt.Chart(cleaned).\
                encode(alt.X("date:N"), 
                       alt.Y("cases", scale=alt.Scale(type="log")), 
                       alt.Color("case type", 
                                 sort=alt.EncodingSortField(field="cases", 
                                                            order="descending", 
                                                            op="max")),
                       tooltip=['date', 'state', 'case type', 'cases']).\
                transform_filter(alt.datum.state == state).\
                transform_filter(alt.datum.cases > 0).\
                transform_filter(alt.FieldOneOfPredicate("case type", case_types))
    
    return chart.mark_line() + chart.mark_point() if show_points else chart.mark_line()


In [7]:
cases_for_state("WI")

alt.Chart(...)

# Interactive chart

In [8]:
def interactive_cases_for_state():
    case_types = ['death', 'positive', 'hospitalizedCumulative', 'inIcuCumulative']
    input_dropdown = alt.binding_select(options=cleaned[(cleaned["cases"] > 0) & (cleaned["case type"] == "positive")]["state"].sort_values().unique())
    selection = alt.selection_single(fields=['state'], bind=input_dropdown, name='Choose', init={"state":"AK"})

    chart = alt.Chart(cleaned).\
                encode(alt.X("date:N"), 
                       alt.Y("cases", scale=alt.Scale(type="log")), 
                       alt.Color("case type", 
                                 sort=alt.EncodingSortField(field="cases", 
                                                            order="descending", 
                                                            op="max")),
                       tooltip=['date', 'state', 'case type', 'cases']).\
                transform_filter(selection).\
                transform_filter(alt.datum.cases > 0).\
                transform_filter(alt.FieldOneOfPredicate("case type", case_types)).\
                add_selection(selection)
    
    return chart.mark_line()


In [9]:
interactive_cases_for_state()

alt.Chart(...)

# Map visualization

In [10]:
states = alt.topo_feature("https://vega.github.io/vega-datasets/data/us-10m.json", "states")


In [11]:
ctrim = cleaned[(cleaned["case type"] == "positive") & (cleaned["date"] == cleaned["date"].max())].copy()

alt.Chart(
    states
    ).mark_geoshape(
    ).encode(
        color='cases:Q',
        tooltip=['state:N', 'cases:Q', 'date:N']
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(ctrim, 'fips', ['cases', 'case type', 'state', 'date'])
    ).project(
        type='albersUsa'
    ).properties(
        width=500, height=400
    )


alt.Chart(...)